In [ ]:
import sys
sys.path.append('../')
import os

DATABASE = os.environ['DATABASE_URL'].split('sqlite:///')[1]

# Env vars for Men / Women's tournament
DATA_PATH = os.environ['DATA_PATH'] + "/" + os.environ["DATA_PREFIX"]
gender = "WOMENS" if os.environ["DATA_PREFIX"] == "W" else "MENS"

from models import Seed, engine, Team, Match, MatchPredictions, SEASON
from elo_run.response_function import home_response, away_response, neutral_response
from elo_run.param_tuning import run_system, set_up_elo_model, link_function_list

import pandas as pd
import datetime
from typing import Dict, Set

from collections import defaultdict

from sqlalchemy.orm import sessionmaker
from sqlalchemy import *

Session = sessionmaker(bind=engine)

session = Session()

In [ ]:
row = """25	-70.0	L	1200.0	15.0	-10.0"""
description = "slow_update_high_seed"

In [ ]:
vals = row.split("	")

In [ ]:
# Define a system, and get match predictions
k = int(vals[0])
seed = float(vals[1])
link = vals[2]
link_function = link_function_list[link]
fgp = float(vals[3])
r = float(vals[4])
fgp3 = float(vals[5])

elo = set_up_elo_model(k=k, seed=seed, link_function=link_function, 
                       fgp=fgp, fgp3=fgp3, r=r)





In [ ]:
match_predictions = run_system(elo, SEASON)
match_predictions.to_sql(
        con=engine, index=False, name=MatchPredictions.__tablename__, if_exists="replace"
    )

In [ ]:
# Get the team id to name
team_id_to_name = {x:y for x,y in session.query(Team.TeamID, Team.TeamName).all()}

In [ ]:
# Get all teams in the tournament for predicting
dancers = pd.DataFrame(session.query(Seed.TeamID, Seed.Seed.label('seed')).filter(Seed.Season==SEASON).all())

dancers_df = dancers.sort_values(by='TeamID').set_index('TeamID', drop = True)
dancers_df = dancers_df.join(pd.DataFrame.from_dict(team_id_to_name, orient='index', 
                                                    columns=['TeamName']), how='left')

In [ ]:
# Get the most recent stats before the tournament
def get_most_recent_stats(Season):
    """
    For a given Season, get the most recent stats available
    :param Season:
    :return:
    """
    Session = sessionmaker(bind=engine)
    session = Session()

    q = session.query(Team.TeamID, func.max(Match.mdid).label('mdid')).join(
    Match.teams).filter(Match.Season == Season).group_by(Team.TeamID).subquery()

    winners = list(session.query(Match.WTeamID, Match.WFGP3_avg, Match.WFGP_avg, Match.WR_avg, MatchPredictions.WTeamRatingAfter).join(
    q, and_(Match.WTeamID==q.c.TeamID, Match.mdid == q.c.mdid)
                                                                       ).join(MatchPredictions).all())

    losers = list(session.query(Match.LTeamID, Match.LFGP3_avg, Match.LFGP_avg, Match.LR_avg, MatchPredictions.LTeamRatingAfter).join(
    q, and_(Match.LTeamID==q.c.TeamID, Match.mdid == q.c.mdid)
                                                                       ).join(MatchPredictions).all())

    all_stats = winners + losers
    
    df = pd.DataFrame(all_stats, columns = ['TeamID', 'FGP3', 'FGP', 'R','rating'])
    df.set_index('TeamID', inplace = True, drop = True)

    return df

dancer_to_stats = get_most_recent_stats(SEASON)

In [ ]:

full_df = dancers_df.merge(dancer_to_stats, left_index = True, right_index = True)

dancers_dicts = full_df.to_dict(orient='index')

In [ ]:
response_fns = {
    'H': home_response,
    'A': away_response,
    'N': neutral_response
}

predictions = []
predictions_named = []

In [ ]:
# Get seeds
df_seeds = pd.read_csv(f"{DATA_PATH}NCAATourneySeeds.csv").query(
    f"Season == {SEASON}").drop(columns=["Season"]).reset_index(drop=True)
seed_to_team_id = df_seeds.set_index("Seed").to_dict()['TeamID']

In [ ]:
first_four_slots = [x for x in seed_to_team_id if 'a' in x]

In [ ]:
first_four_slots

In [ ]:
# First Four results -> TO BE UPDATED

for i in first_four_slots:
    other = i.replace("a", "b")
    print(i, 
          f"{seed_to_team_id[i]}:{team_id_to_name[seed_to_team_id[i]]}",
            f"{seed_to_team_id[other]}:{team_id_to_name[seed_to_team_id[other]]}",
         )

In [ ]:
# 2023 Results
if gender == "MENS":
    seed_to_team_id['W16'] = 1192
    seed_to_team_id['X16'] = 1394
    seed_to_team_id['Y11'] = 1338
    seed_to_team_id['Z11'] = 1113
    remaining_first_four_seeds = []
    remaining_first_four_teams = []
    losers = [1411, 1369, 1280, 1305]
else:
    seed_to_team_id['W11'] = 3280
    seed_to_team_id['X16'] = 3357
    remaining_first_four_seeds = ["Y16", "Z11"]
    losers = [3228, 3380]
    remaining_first_four_teams = [3284, 3399, 3345, 3385]

In [ ]:
print(remaining_first_four_seeds)

In [ ]:
# Get the mapping of round to potential opponents

df_slots = pd.read_csv(f"{DATA_PATH}NCAATourneySlots.csv").query(
    f"Season == {SEASON}").drop(columns=["Season"]).reset_index(drop=True)


df_slots['round'] = df_slots["Slot"].apply(lambda x: int(x[1]) if x[0] == "R" else 0)


slot_to_teams:Dict[str,Set[str]] = dict()
round_to_team_to_opponents:Dict[int,Dict[str,Set[str]]] = defaultdict(
    lambda: defaultdict(set))

# For each slot get the possible opponents
# Slot = e.g. R1W2 -> Round 1, the match which W02 seed would play 
# in as the strong seed

for _, row in df_slots.sort_values(by=['round']).iterrows():
    slot = row['Slot']
    if 'R' in slot or slot in remaining_first_four_seeds:
        rd = row['round']

        strong = row['StrongSeed']  # e.g. W02 or R1W2

        # The teams feeding into the strong seed's previous match
        weak_opponents = slot_to_teams.get(strong, {strong})

        # Get the weak sided equivalent
        weak = row['WeakSeed']
        strong_opponents = slot_to_teams.get(weak, {weak})

        # Update register
        for wk in weak_opponents:
            for st in strong_opponents:
                round_to_team_to_opponents[rd][st].add(wk)
                round_to_team_to_opponents[rd][wk].add(st)


        teams = strong_opponents.union(weak_opponents)
        slot_to_teams[slot]= teams

In [ ]:
losers

In [ ]:
# Set up team ratings
round_to_team_id_to_rating: Dict[int,Dict[int,float]] = defaultdict(lambda: defaultdict(float))
for team_id, dd in dancers_dicts.items():
    if team_id in remaining_first_four_teams:
        r = 0
    else:
        r = 1
    round_to_team_id_to_rating[r][team_id] = dd['rating']
    
# Set up record of probability of a team reaching round N
round_to_team_id_to_prob: Dict[int,Dict[int,float]] = defaultdict(lambda: defaultdict(float))

for team_id, dd in dancers_dicts.items():
    if team_id in remaining_first_four_teams:
        p = 1
        r = 0
    elif team_id in losers:
        p = 0
        r = 1
    else:
        p = 1
        r = 1
    round_to_team_id_to_prob[r][team_id] = p

    
# All 6 championship rounds
for rd in range(0,7):
    # Get the matches for the round
    rd_matches = round_to_team_to_opponents[rd]
    
    # For each team and set of opponents
    for team, opponents in rd_matches.items():
        # Only do once per matchup
        for opponent in opponents:
            if team < opponent:
                
                # Set team ids
                team_id = seed_to_team_id[team]
                opponent_id = seed_to_team_id[opponent]

                # Set team 1 and team 2 by using min team id as team 1
                if team_id < opponent_id:
                    team_1 = team_id
                    team_2 = opponent_id
                    team_1_seed = team
                    team_2_seed = opponent
                else:
                    team_2 = team_id
                    team_1 = opponent_id
                    team_2_seed = team
                    team_1_seed = opponent

                # Get current ratings
                team_1_rating = round_to_team_id_to_rating[rd][team_1]
                team_2_rating = round_to_team_id_to_rating[rd][team_2]
                
                # Get stats and latest rating
                team_1_stats = {**dancers_dicts[team_1], **{'rating': team_1_rating}}
                team_2_stats = {**dancers_dicts[team_2], **{'rating': team_2_rating}}
                
                # Predict matchup
                prediction = elo.predict(team_1_stats, team_2_stats)
                
                # Save prediction
                matchup_id = f"{SEASON}_{team_1}_{team_2}"

                # Update ratings probabilistically
                if rd < 6:
                    
                    # Assume point win of 8
                    point_diff = 8
                    result_likelihood = max(elo.response(point_diff, 'N'), prediction + 0.02)
                    # Update team 1's rating if they had beaten this opponent
                    team_1_new = elo.update(prediction, result_likelihood, team_1_rating, elo.K)
                    
                    prob_playing_opponent = round_to_team_id_to_prob[rd][team_2]
                        
                    # The expected rating at this stage should be the new rating
                    # multiplied the probability this matchup occurred
                    round_to_team_id_to_rating[rd+1][team_1] += team_1_new * prob_playing_opponent
                    # Prob of making it to next round is the prob they made it to this round
                    # X by prob they win this match X prob opponent was their opponent
                    round_to_team_id_to_prob[rd+1][team_1] += (prediction * prob_playing_opponent
                                                              ) * round_to_team_id_to_prob[rd][team_1]

                    # Repeat for other team
                    point_diff = 8
                    result_likelihood = max(elo.response(point_diff, 'N'), 1-prediction + 0.02)
                    team_2_new = elo.update(1-prediction, result_likelihood, team_2_rating, elo.K)

                    
                    prob_playing_opponent = round_to_team_id_to_prob[rd][team_1]
                    
                    round_to_team_id_to_rating[rd+1][team_2] += team_2_new * prob_playing_opponent
                    round_to_team_id_to_prob[rd+1][team_2] += ((1-prediction) * prob_playing_opponent
                                                              ) * round_to_team_id_to_prob[rd][team_2]
                    

                predictions.append((matchup_id
                                    , prediction))
                predictions_named.append((team_1_stats['TeamName'], team_2_stats['TeamName']
                                    , prediction))

In [ ]:
# Check out a top seed for sanity
team_id = seed_to_team_id['Z01']

In [ ]:
dancers_dicts[team_id]

In [ ]:
for rd in [1,2,3,4,5,6]:
    print(round_to_team_id_to_prob[rd][team_id])

In [ ]:
team_to_seed = {v:k for k,v in seed_to_team_id.items()}

In [ ]:
branch = "Y"
teams = {v for k,v in seed_to_team_id.items() if branch in k}
for r, item in round_to_team_id_to_prob.items():
    r_total = 0
    for t, p in item.items():
        if t in teams:
            if r == 1:
                print(team_to_seed[t],p)
#             print(r, p)
            r_total += p
    print(r, r_total)

In [ ]:
sample_df = pd.read_csv(f"{DATA_PATH[:-2]}/SampleSubmission{SEASON}.csv")

# Womens
if gender == "WOMENS":
    sample_submission_rows = sample_df.query(f"ID > '{SEASON}_3'")
# Mens
else:
    sample_submission_rows = sample_df.query(f"ID < '{SEASON}_3'")

prediction_df = pd.DataFrame(predictions, columns = ['ID', 'Pred'])

prediction_ids = set(prediction_df['ID'])

sample_submission_rows = sample_submission_rows[
    ~sample_submission_rows["ID"].isin(prediction_ids)]

In [ ]:
# Save submissions
final_out_df = pd.concat([prediction_df, sample_submission_rows], ignore_index=True)
timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")
final_out_df.to_csv(f'../submissions/ids/{gender[0]}_{description}_{timestamp}_{k}_{seed}_{link}_{fgp}_{fgp3}_{r}.csv', index=False)

prediction_named_df = pd.DataFrame(predictions_named, columns = ['Team1', 'Team2', 'Pred'])

prediction_named_df.to_csv(f'../submissions/names/{gender[0]}_{description}_{timestamp}_{k}_{seed}_{link}_{fgp}_{fgp3}_{r}.csv', index=False)

In [ ]:
# Combine with womens